# Noise Analysis with Cirq

This notebook demonstrates how devqubit tracks **noise characterization experiments**.

**What you'll see:**
- GHZ circuit fidelity under different noise models
- Noise model sweep (depolarizing, amplitude damping, bit flip)
- Fidelity vs. circuit depth scaling
- Distribution comparison using TVD

**The workflow:** We establish an ideal (noiseless) baseline, then systematically study how different noise sources affect quantum state preparation.

In [ ]:
from importlib.metadata import entry_points

if not any(ep.name == "cirq" for ep in entry_points(group="devqubit.adapters")):
    raise ImportError(
        "devqubit Cirq adapter is not installed.\n"
        "Install with: pip install 'devqubit[cirq]'"
    )
else:
    print("Cirq adapter available!")

In [ ]:
from pathlib import Path
import shutil
import numpy as np
import cirq

from devqubit import Config, set_config, track
from devqubit.compare import diff
from devqubit.runs import load_run, list_runs, list_runs_in_group, list_groups

### Setup

We use a 5-qubit GHZ circuit as our benchmark. GHZ states are maximally entangled and highly sensitive to noise — perfect for characterization experiments.

**GHZ state:** $|\text{GHZ}\rangle = \frac{1}{\sqrt{2}}(|00000\rangle + |11111\rangle)$

In [ ]:
# Configuration
PROJECT = "cirq_noise_analysis"
N_QUBITS = 5
SHOTS = 4096
SEED = 42

# Workspace
WORKSPACE = Path(".devqubit_cirq_demo")
if WORKSPACE.exists():
    shutil.rmtree(WORKSPACE)

set_config(Config(root_dir=WORKSPACE))

np.random.seed(SEED)
print(f"Workspace: {WORKSPACE.resolve()}")

### 1. Circuit and Simulation Utilities

We define:
- **GHZ circuit builder**: H gate followed by CNOT chain
- **Noise model builder**: wraps simulator with depolarizing, amplitude damping, or bit flip noise
- **Fidelity metric**: measures how often we get the ideal |00...0⟩ or |11...1⟩ outcomes

In [ ]:
def create_ghz_circuit(n_qubits: int) -> cirq.Circuit:
    """Create n-qubit GHZ circuit."""
    qubits = cirq.LineQubit.range(n_qubits)
    circuit = cirq.Circuit()
    circuit.append(cirq.H(qubits[0]))
    for i in range(n_qubits - 1):
        circuit.append(cirq.CNOT(qubits[i], qubits[i + 1]))
    circuit.append(cirq.measure(*qubits, key="m"))
    return circuit


def build_noisy_simulator(noise_type: str, error_rate: float):
    """Create simulator with specified noise model."""
    if noise_type == "ideal":
        return cirq.Simulator()

    noise_channels = {
        "depolarizing": cirq.depolarize(error_rate),
        "amplitude_damping": cirq.amplitude_damp(error_rate),
        "bit_flip": cirq.bit_flip(error_rate),
    }
    noise_model = cirq.ConstantQubitNoiseModel(noise_channels[noise_type])
    return cirq.DensityMatrixSimulator(noise=noise_model)


def run_circuit(sim, circuit: cirq.Circuit, shots: int) -> dict:
    """Run circuit and return measurement counts."""
    result = sim.run(circuit, repetitions=shots)
    measurements = result.measurements["m"]
    bitstrings = ["".join(map(str, row)) for row in measurements]
    unique, counts = np.unique(bitstrings, return_counts=True)
    return dict(zip(unique, counts.tolist()))


def ghz_fidelity(counts: dict, n_qubits: int, shots: int) -> float:
    """Compute GHZ fidelity: P(|00...0⟩) + P(|11...1⟩)."""
    ideal_0 = "0" * n_qubits
    ideal_1 = "1" * n_qubits
    p0 = counts.get(ideal_0, 0) / shots
    p1 = counts.get(ideal_1, 0) / shots
    return p0 + p1  # Ideal GHZ should be ~1.0

In [ ]:
# Build and display the circuit
circuit = create_ghz_circuit(N_QUBITS)

print(f"GHZ-{N_QUBITS} Circuit:")
print(circuit)

### 2. Ideal Baseline

First, we establish the **ideal (noiseless) baseline**. This gives us the reference distribution to compare against.

For a perfect GHZ state, we expect:
- ~50% probability of |00000⟩
- ~50% probability of |11111⟩
- Fidelity ≈ 1.0

In [ ]:
with track(project=PROJECT, run_name="ideal_baseline") as run:
    sim = run.wrap(cirq.Simulator())

    run.log_params(
        {
            "n_qubits": N_QUBITS,
            "shots": SHOTS,
            "noise_type": "ideal",
            "error_rate": 0.0,
        }
    )
    run.set_tags({"role": "baseline", "noise": "ideal"})

    counts = run_circuit(sim, circuit, SHOTS)
    fidelity = ghz_fidelity(counts, N_QUBITS, SHOTS)

    run.log_metrics({"fidelity": fidelity, "unique_outcomes": len(counts)})
    run.log_json("counts", counts, role="result")

    ideal_run_id = run.run_id

print(f"Ideal baseline: {ideal_run_id}")
print(f"Fidelity: {fidelity:.4f}")
print(f"Top outcomes: {sorted(counts.items(), key=lambda x: -x[1])[:3]}")

### 3. Noise Model Sweep

Now we compare how different noise models affect the GHZ fidelity:

| Noise Type | Physical Origin | Effect |
|------------|-----------------|--------|
| **Depolarizing** | Generic decoherence | Random Pauli errors |
| **Amplitude damping** | T1 relaxation | Decay toward |0⟩ |
| **Bit flip** | Classical errors | Flips qubit state |

All runs are grouped under `noise_sweep` for easy comparison.

In [ ]:
noise_configs = [
    {"type": "depolarizing", "rate": 0.01},
    {"type": "depolarizing", "rate": 0.02},
    {"type": "amplitude_damping", "rate": 0.02},
    {"type": "bit_flip", "rate": 0.01},
]

sweep_results = []

print("Noise Model Sweep")
print("=" * 55)

for cfg in noise_configs:
    with track(
        project=PROJECT,
        group_id="noise_sweep",
        group_name="Noise Model Comparison",
    ) as run:
        sim = run.wrap(build_noisy_simulator(cfg["type"], cfg["rate"]))

        run.log_params(
            {
                "n_qubits": N_QUBITS,
                "shots": SHOTS,
                "noise_type": cfg["type"],
                "error_rate": cfg["rate"],
            }
        )
        run.set_tags({"role": "sweep", "noise": cfg["type"]})

        counts = run_circuit(sim, circuit, SHOTS)
        fidelity = ghz_fidelity(counts, N_QUBITS, SHOTS)

        run.log_metrics({"fidelity": fidelity, "unique_outcomes": len(counts)})
        run.log_json("counts", counts, role="result")

        sweep_results.append(
            {
                "noise_type": cfg["type"],
                "error_rate": cfg["rate"],
                "fidelity": fidelity,
                "run_id": run.run_id,
            }
        )

        print(f"  {cfg['type']:18s}  rate={cfg['rate']:.3f}  fidelity={fidelity:.4f}")

### Interpretation

Let's rank the noise models by their impact on fidelity. Lower fidelity means the noise is more destructive to the GHZ state.

In [ ]:
print("Fidelity Ranking (highest => lowest):")
print("-" * 45)

for r in sorted(sweep_results, key=lambda x: x["fidelity"], reverse=True):
    loss = 1 - r["fidelity"]
    print(f"  {r['noise_type']:18s}  rate={r['error_rate']:.3f}  loss={loss:.4f}")

### 4. Fidelity vs. Circuit Depth

Noise accumulates with circuit depth. Let's see how fidelity degrades as we make the circuit deeper.

We create deeper GHZ-like circuits by adding extra entanglement layers (forward and backward CNOT chains).

In [ ]:
def create_deep_ghz(n_qubits: int, extra_layers: int) -> cirq.Circuit:
    """Create deeper GHZ-like circuit with extra entanglement layers."""
    qubits = cirq.LineQubit.range(n_qubits)
    circuit = cirq.Circuit()

    # Initial GHZ
    circuit.append(cirq.H(qubits[0]))

    # Extra layers: forward + backward CNOT chains
    for _ in range(extra_layers):
        for i in range(n_qubits - 1):
            circuit.append(cirq.CNOT(qubits[i], qubits[i + 1]))
        for i in range(n_qubits - 2, -1, -1):
            circuit.append(cirq.CNOT(qubits[i], qubits[i + 1]))

    # Final CNOT chain to restore GHZ
    for i in range(n_qubits - 1):
        circuit.append(cirq.CNOT(qubits[i], qubits[i + 1]))

    circuit.append(cirq.measure(*qubits, key="m"))
    return circuit

In [ ]:
depth_results = []

# Use moderate depolarizing noise
noisy_sim = build_noisy_simulator("depolarizing", 0.005)

print("Depth Scaling (depolarizing noise, p=0.005)")
print("=" * 50)

with track(
    project=PROJECT,
    group_id="depth_sweep",
    group_name="Fidelity vs Depth",
) as run:
    run.log_params(
        {
            "n_qubits": 4,
            "noise_type": "depolarizing",
            "error_rate": 0.005,
        }
    )
    run.set_tag("experiment", "depth_scaling")

    for extra_layers in [0, 1, 2, 4, 8]:
        circ = create_deep_ghz(4, extra_layers)
        depth = len(circ) - 1  # Exclude measurement

        counts = run_circuit(noisy_sim, circ, 2048)
        fid = ghz_fidelity(counts, 4, 2048)

        # Log as time-series
        run.log_metric("fidelity", fid, step=extra_layers)
        run.log_metric("depth", depth, step=extra_layers)

        depth_results.append(
            {"extra_layers": extra_layers, "depth": depth, "fidelity": fid}
        )
        print(f"  layers={extra_layers}  depth={depth:3d}  fidelity={fid:.4f}")

    depth_run_id = run.run_id

### Exponential Decay

You should see fidelity decay roughly exponentially with depth. This is characteristic of depolarizing noise accumulation.

### 5. Compare Ideal vs. Noisy

Let's compare the ideal baseline with a noisy run. The `diff()` function will show:
- Parameter differences
- Metric differences
- TVD (Total Variation Distance) between distributions

In [ ]:
# Find the depolarizing run with rate=0.02
depol_run = next(
    r
    for r in sweep_results
    if r["noise_type"] == "depolarizing" and r["error_rate"] == 0.02
)

comparison = diff(ideal_run_id, depol_run["run_id"])

print("Ideal vs. Depolarizing (2% error)")
print("=" * 50)
print(comparison)

### 6. Experiment Summary

Browse all runs and groups from this analysis.

In [ ]:
print("All Noise Analysis Runs")
print("=" * 65)

for info in list_runs(project=PROJECT):
    rec = load_run(info["run_id"])
    noise = rec.params.get("noise_type", "?")
    rate = rec.params.get("error_rate", 0)
    fidelity = rec.metrics.get("fidelity", 0)
    role = rec.tags.get("role", "-")

    print(
        f"  {rec.run_id[:12]}  {role:8s}  {noise:18s}  rate={rate:.3f}  fid={fidelity:.4f}"
    )

In [ ]:
print("\nExperiment Groups")
print("-" * 40)

for g in list_groups():
    runs = list_runs_in_group(g["group_id"])
    print(f"  {g['group_name']}: {len(runs)} runs")

---
## Summary

| Feature | What we demonstrated |
|---------|----------------------|
| **Ideal baseline** | Reference for comparison |
| **Noise sweep** | Compare depolarizing, amplitude damping, bit flip |
| **Depth scaling** | Fidelity decay vs. circuit depth |
| **Run comparison** | TVD between ideal and noisy distributions |

**Key insight:** devqubit tracks noise characterization workflows with the same patterns as optimization. Parameters, metrics, and artifacts all work the same way.

In [ ]:
shutil.rmtree(WORKSPACE)
print("Done!")